# Varying Node Capacities

Evaluation of solution quality for abilene network with increasing node capacities.

## Scenario

* Network: Abilene: 11 nodes, 14 edges
* All nodes with random cap between (0-2)*(1,2,3,4,5,6,7)
* 4 Ingress Nodes
* MMPP flow arrival with inter-arr time 8-12
* Flow weight = 1, Delay weight = 0

## Approaches

* BSP: 30 repetition per scenario
* Load Balance: 30 repetition per scenario
* Shortest Path: 30 repetition per scenario
* DRL: 
  * 100k training  with 10 different seeds on larger networks --> auto select best one
  * 30x 1 testing episode with best agent

In [1]:
import os
import pandas as pd
import yaml
from operator import itemgetter
from collections import deque
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx
%matplotlib inline

os.makedirs('plots', exist_ok=True)
# prefix for saving plot files
plot_prefix = 'var-node-cap_mmpp'

/home/dat/git_folder_edge_4/realVNF/venv/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
# adjusted version
def read_results_over_networks(dir, df=None, rf=None, best_only=False, network_dir=None, skip_rf=True, alg_name=None):
    """Read result files in the directory and append results to the data frame"""
    if df is None:
        df = pd.DataFrame()
    if rf is None:
        rf = pd.DataFrame()

    if network_dir is not None:
        dir = f'{network_dir}/{dir}'
    
    for root, sub_dirs, files in os.walk(dir):
        if len(files) > 0 and (not '.ipynb_checkpoints' in root):                
            if 'best' in root or not best_only:
                # read metrics at end of simulation (last row)
                df_metrics = pd.read_csv(f'{root}/metrics.csv')
                data = df_metrics.tail(1)
                # get the last placement
                df_placements = pd.read_csv(f'{root}/placements.csv').groupby('time')
                placements = pd.concat(deque(map(itemgetter(1), df_placements), maxlen=1))
                
                
                # add algorithm name, number of ingress nodes, and number of vnfs as columns
                with open(f'{root}/input.yaml', 'r') as f:
                    inputs = yaml.load(f, Loader=yaml.Loader)
                if alg_name is None:
                    alg_name = inputs['algorithm']
                
                # read number of nodes in network
                network_file = [filename for filename in files if ".graphml" in filename]
                assert len(network_file) == 1
                network_file = network_file[0]
                if network_dir is None:
                    network_dir = network_file.split('.')[0]
                nx_network = networkx.read_graphml(f'{root}/{network_file}')
                num_nodes = nx_network.number_of_nodes()
                network_cap = sum([n[1].get('NodeCap') for n in nx_network.nodes(data=True)])
                
                data.insert(loc=0, column='network', value=network_dir)
                data.insert(loc=1, column='network_cap', value=network_cap)
                data.insert(loc=2, column='algorithm', value=alg_name)
                data.insert(loc=3, column='num_nodes', value=num_nodes)
                data.insert(loc=4, column='num_ingress', value=inputs['num_ingress'])
                data.insert(loc=data.columns.size, column='num_vnfs', value=placements['sf'].size)
                
                df = df.append(data, ignore_index=True, sort=True)
                fix_dtypes(df)
                df.sort_values(['algorithm', 'num_nodes'], inplace=True)
                df = df[['network', 'algorithm', 'num_nodes', 'num_ingress', 'network_cap', 'total_flows', 'successful_flows', 'dropped_flows', 'in_network_flows', 'avg_end2end_delay', 'num_vnfs']]

                if not skip_rf:
                    # Reading resource metrics and creating a new Resource Dataframe
                    # Getting the last group using the Deque operation
                    rf_resources = pd.read_csv(f'{root}/node_metrics.csv').groupby('time')
                    resources = pd.concat(deque(map(itemgetter(1), rf_resources), maxlen=1))
                    resources.insert(loc=0, column='network', value=network_dir)
                    resources.insert(loc=1, column='network_cap', value=network_cap)
                    resources.insert(loc=2, column='algorithm', value=inputs['algorithm'])
                    resources.insert(loc=3, column='num_nodes', value=num_nodes)
                    resources.insert(loc=4, column='num_ingress', value=inputs['num_ingress'])
                    rf = rf.append(resources, ignore_index=True, sort=True)
                    rf.sort_values(['algorithm', 'num_nodes'], inplace=True)
                
    #Add percentage of successful flows
    succ_percentage = []
    for index, row in df.iterrows():
        succ_percentage.append(row['successful_flows'] / row['total_flows'] * 100)
    df['successful_percentage'] = succ_percentage
    return df, rf


def fix_dtypes(df):
    """Set correct dtypes and order"""
    df['avg_end2end_delay'] = pd.to_numeric(df['avg_end2end_delay'])
    df['total_flows'] = pd.to_numeric(df['total_flows'])
    df['successful_flows'] = pd.to_numeric(df['successful_flows'])
    df['dropped_flows'] = pd.to_numeric(df['dropped_flows'])
    df['in_network_flows'] = pd.to_numeric(df['in_network_flows'])
    
    
def df_mean_std(df, group_by=['network', 'algorithm', 'num_nodes', 'num_ingress' , 'network_cap']):
    """Return 2 new dfs with 1) average and 2) std values"""
    df_mean = df.groupby(group_by).mean().reset_index()
    df_std = df.groupby(group_by).std().reset_index()
    return df_mean, df_std

In [4]:
# read results
df = pd.DataFrame()
rf = pd.DataFrame()

df, rf = read_results_over_networks('drl', df, rf, best_only=True, alg_name='DRL')
df, rf = read_results_over_networks('bsp', df, rf, best_only=False, alg_name='BSP')
df, rf = read_results_over_networks('bsp-ad', df, rf, best_only=False, alg_name='BSP Ad.')
df, rf = read_results_over_networks('sp', df, rf, best_only=False, alg_name='SP')
df, rf = read_results_over_networks('lb', df, rf, best_only=False, alg_name='LB')

df_mean, df_std = df_mean_std(df)
# only select results with 4 ingress nodes
df_mean = df_mean[df_mean['num_ingress'] == 4]
df_std = df_std[df_std['num_ingress'] == 4]

df_mean

KeyError: 'network'

## Successful Flows

In [ ]:
sns.set(font_scale=1.1, style='white')

In [ ]:
def plot_successful_flows(df_mean, df_std, filename=None):
    algs=['DRL', 'BSP','BSP Ad.', 'SP', 'LB']
    sns.set(font_scale=1, style='white')
    markers = ['^', 'v', 's', 'D', '<', '>', 'P', 'X', 'o', '8', 'p']
    fig, ax1 = plt.subplots(figsize=(4, 4))
    sns.set(font_scale=1.1, style='white')

    # plot successful flows
    for i, algorithm in enumerate(algs):
        alg_df_mean = df_mean[df_mean['algorithm'] == algorithm]
        alg_df_std = df_std[df_std['algorithm'] == algorithm]
        ax1.errorbar(alg_df_mean['network_cap'], alg_df_mean['successful_percentage'], yerr=alg_df_std['successful_percentage'], capsize=5,
                    label='{}'.format(algorithm), marker=markers[i])

    # set axis, title, legend
    # ax1.set_title('Successful Flows')
    ax1.set_xlabel('Total Node Capacity')
    ax1.set_ylabel('Successful Flows [%]')
    ax1.set_ylim(0, 105)

    ax1.tick_params(axis='both', direction='inout', length=5, bottom=True, left=True, right=True, top=True)

    # remove error bars from legend: https://stackoverflow.com/a/15551976/2745116
    # get handles
    handles, labels = ax1.get_legend_handles_labels()
    # remove the errorbars
    handles = [h[0] for h in handles]
    # use them in the legend
    ax1.legend(handles, labels, numpoints=1)

    # ax1.legend()
    if filename is not None:
        plt.tight_layout()
        fig.savefig(f'plots/{filename}')

In [ ]:
plot_successful_flows(df_mean, df_std)

